---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [47]:
import json
"""
This section imports the file to assert the keys and returns the output of each key. 
>>>('Organ Failure History' in y.keys())
True
>>>('Age' in y.keys())
True 
"""
with open('apache-final.json') as x:
    y=json.load(x)

In [48]:
#Test functions
('Organ Failure History' in y.keys())

True

In [49]:
('Age' in y.keys())


True

In [50]:
assert('Age' in y.keys())
assert('Temperature' in y.keys())
assert('pH' in y.keys())
assert('Heart Rate' in y.keys())
assert('Respiratory Rate' in y.keys())
assert('Sodium' in y.keys())
assert('Potassium' in y.keys())
assert('Creatinine' in y.keys())
assert('Hematocrit' in y.keys())
assert('White Blood Count' in y.keys())
assert('FiO2' in y.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

In [51]:
import json
### BEGIN SOLUTION for first function risk factors

def risk_factors_score(risks_list, config_file):
    """
    This function evalautes the risks in the apache formula and returns the score per risk according to the APACHE site. 
    The test functions are below. 
    >>>(risk_factors_score(['Nonoperative'],RISK_FACTORS_CONFIG_FILE)
    5
    """
    
    score = 0
    
    
    
    config = json.load(open(config_file))
    risk_factor_scores = config.get("Organ Failure History")
    
    
    for risk in risks_list:
        if risk in risk_factor_scores:
            score += risk_factor_scores.get(risk)
    
    
    ### END SOLUTION
   
    return score

In [52]:
RISK_FACTORS_CONFIG_FILE = "apache-final.json"

In [53]:
assert(risk_factors_score([],RISK_FACTORS_CONFIG_FILE) == 0)
assert(risk_factors_score(['Nonoperative'],RISK_FACTORS_CONFIG_FILE) == 5)
assert(risk_factors_score(['Emergency'],RISK_FACTORS_CONFIG_FILE) == 5)

In [62]:
### Second function #####

import json
"""
    This function evalautes the age, temperature in Celsius, and pH levels. The reason this function includes 
        these parameters because they are closely related to each other since it gathers
        basic information from the patient/tester. The reason pd.Series is added because in the final function pandas is used.
        This overides the error "cannot convert the series to <class 'float'>", 'occurred at index loss').
        >>>all_factors(6,0,0, AGE_TEMP_PH_CONFIG_FILE)
        8
    """
def all_factors(age, temp, ph, config_file):
    
    score=0
    config=json.load(open(config_file))
    age_rules= config.get("Age")  
    temp_rules=config.get("Temperature")
    ph_rules=config.get("pH")
    
    for rule in age_rules:
        if pd.Series((age) >= rule.get('min')).all() and pd.Series((age) < rule.get('max')).all():
            score= rule.get('points')
    
    score2=0
    
    for rule2 in temp_rules:
        if pd.Series((temp) >= rule2.get('min')).all() and pd.Series((temp) < rule2.get('max')).all():
            score2= rule2.get('points')
    score3=0
    for rule3 in ph_rules:
        if pd.Series((ph) >= rule3.get('min')).all() and pd.Series((ph) < rule3.get('max')).all():
            score3=rule3.get('points')
    final_score=score+score2+score3    
    
    return final_score



In [63]:
AGE_TEMP_PH_CONFIG_FILE = "apache-final.json"



In [64]:
assert(all_factors(6,0,0, AGE_TEMP_PH_CONFIG_FILE))==8
assert(all_factors(56,44,7, AGE_TEMP_PH_CONFIG_FILE)==11)
assert(all_factors(0,0,0, AGE_TEMP_PH_CONFIG_FILE)==8)

In [65]:
###Third function ####

import json
    
def rates(heart, respiratory, config_file):
    """
        This function evaluates the heart and respiratory rate for the patient. It is the same logic as above but place as its
        own function since it is a different category. Ths function returns a score for the patient. 
        >>>(rates(42,8, HEART_RESP_CONFIG_FILE))
        5
    """
    score4=0
    config=json.load(open(config_file))
      
    heart_rules=config.get("Heart Rate")
    resp_rules= config.get("Respiratory Rate")  
    
    for rule4 in heart_rules:
        if pd.Series((heart) >= rule4.get('min')).all() and pd.Series((heart) < rule4.get('max')).all():
            score4= rule4.get('points')
    
    score5=0
    
    for rule5 in resp_rules:
        if pd.Series((respiratory) >= rule5.get('min')).all() and pd.Series((respiratory) < rule5.get('max')).all():
            score5= rule5.get('points')
    
    final_score_rates=score4+score5   
    
    return final_score_rates

In [66]:
HEART_RESP_CONFIG_FILE = "apache-final.json"


In [67]:
rates(100,79,HEART_RESP_CONFIG_FILE)

4

In [68]:
assert(rates(42,8, HEART_RESP_CONFIG_FILE))==5
assert(rates(6,10, HEART_RESP_CONFIG_FILE))==6
assert(rates(100,79, HEART_RESP_CONFIG_FILE))==4

In [69]:
### Forth function###
import json
"""
    This function evalautes the sodium and potassium levels of the patient. This function returns a score for the sodium and potassium levels.
    It is the same logic as above but place as its
    own function since it is a different category.
    >>>(serum(0,0, SODIUM_POTASSIUM_CONFIG_FILE))
    8
"""
def serum(sodium, potassium, config_file):

    score6=0
    config=json.load(open(config_file))
    sodium_rules=config.get("Sodium")
    pot_rules=config.get("Potassium")
    
    for rule6 in sodium_rules:
        if pd.Series((sodium) >= rule6.get('min')).all() and pd.Series((sodium) < rule6.get('max')).all():
            score6=rule6.get('points')
        
    score7=0
    for rule7 in pot_rules:
        if pd.Series((potassium) >= rule7.get('min')).all() and pd.Series((potassium) < rule7.get('max')).all():
            score7=rule7.get('points')
    
    final_score_serum=score6+score7
    return final_score_serum
    

In [70]:
SODIUM_POTASSIUM_CONFIG_FILE="apache-final.json"


In [71]:
serum(131,131, SODIUM_POTASSIUM_CONFIG_FILE)

4

In [72]:
assert(serum(0,0, SODIUM_POTASSIUM_CONFIG_FILE))==8
assert(serum(6,15, SODIUM_POTASSIUM_CONFIG_FILE))==8
assert(serum(131,131, SODIUM_POTASSIUM_CONFIG_FILE))==4

In [73]:
###This is the fifth function ###
import json
def blood(white_blood_count, hematocrit, config_file):
    """
    This function returns a score for both a white blood count and hematocrit. Uses the same logic as above howevever it is in a different category
    and has its own function. 
    >>>blood(22,0, WHITE_BLOOD_HEMATOCRIT_CONFIG_FILE))
    6   
    """
    
    config=json.load(open(config_file))
    hemato_1=config.get("Hematocrit")
    white_blood=config.get("White Blood Count")
    
    
    score9=0
    for rule9 in hemato_1:
        if pd.Series((hematocrit) >= rule9.get('min')).all() and pd.Series((hematocrit) < rule9.get('max')).all():
            score9=rule9.get('points')
    score8=0
    for rule8 in white_blood:
        if pd.Series((white_blood_count) >= rule8.get('min')).all() and pd.Series((white_blood_count) < rule8.get('max')).all():
            score8=rule8.get('points')
   
    
    final_score_blood=score8+score9
    return final_score_blood

In [74]:
WHITE_BLOOD_HEMATOCRIT_CONFIG_FILE="apache-final.json"

In [75]:
(blood(5.6,100, WHITE_BLOOD_HEMATOCRIT_CONFIG_FILE))

4

In [76]:
assert(blood(22,0, WHITE_BLOOD_HEMATOCRIT_CONFIG_FILE))==6
assert((blood(55,15, WHITE_BLOOD_HEMATOCRIT_CONFIG_FILE)))==8
assert(blood(5.6,100, WHITE_BLOOD_HEMATOCRIT_CONFIG_FILE))==4

In [77]:
### This function is the sixth function###
"""
This function evaluates the Glasgow score by taking the given score and subtracting it by 15. Equation used 15-glasgow score. 
>>>(glas(13))
2
"""
def glas(points):
    glas_points=15-points

    return glas_points


In [78]:
(glas(13))==2

True

In [79]:
(glas(10))

5

In [80]:
(glas(5))==10

True

In [128]:
### This is the 7th function ###
import json
def muscle(creatinine, failure, config_file):
    """
    This function evaluates the creatinine and whether the patient has acute and chronic renal failure. Ths function uses a if-else statement
    and once that failure is determine then the score can be evaluated. 
    >>>(muscle(3.5,'Chronic Renal Failure', CREATININE_FAILURE_CONFIG_FILE))
    4
    """
    
    mus_score=0
    config = json.load(open(config_file))
    mus_entries = config.get("Creatinine")
    new_mus=mus_entries.get("Acute Renal Failure")
    new_mus2=mus_entries.get("Chronic Renal Failure")
    
    if pd.Series(failure==("Acute Renal Failure")).all():
            creat=config.get("Acute Renal Failure")
            for x in new_mus:
                if pd.Series((creatinine) >= x.get('min')).all() and pd.Series((creatinine) < x.get('max')).all():
                    mus_score= x.get('points')
    elif pd.Series(failure==("Chronic Renal Failure")).all():
             creat2=config.get("Creatinine").get("Chronic Renal Failure")
             for y in new_mus2:
                if float(creatinine) >= y.get('min') and float(creatinine) < y.get('max'):
                    mus_score= y.get('points')
    
    return mus_score

In [129]:
CREATININE_FAILURE_CONFIG_FILE="apache-final.json"

In [130]:
muscle(1.6,['Chronic Renal Failure'], CREATININE_FAILURE_CONFIG_FILE) #with brackets changes sum 

0

In [131]:
assert(muscle(3.5,'Chronic Renal Failure', CREATININE_FAILURE_CONFIG_FILE)==4 )
assert(muscle(1.8,'Chronic Renal Failure', CREATININE_FAILURE_CONFIG_FILE)==2)
assert(muscle(1.6,'Acute Renal Failure', CREATININE_FAILURE_CONFIG_FILE)==4)

In [132]:
muscle(1.6,'Chronic Renal Failure', CREATININE_FAILURE_CONFIG_FILE)

2

In [86]:
###This is the 8th function###
import json 
"""
    This function evaluates the oxygent levels PIO2 and FIO2 and the Aa-gradient. This depends what the patient/tester inputs. This function has
    its own function because the category is related to oxygen. 
    >>>(oxygen(3.5, 20, 400, FIO2_PAO2_GRADIENT_CONFIG_FILE))
    3
"""
def oxygen(FIO2,PAO2, gradient, config_file):
    config=json.load(open(config_file))
    oxygen_entry=config.get("FiO2")
    
    score_ox=0
    
    new_breath=config.get("PaO2")
    new_breath_2=config.get("A-a gradient")
    
    if pd.Series(FIO2 >=oxygen_entry[0].get('min')).all() and pd.Series((FIO2) < oxygen_entry[0].get('max')).all():
                 PAO2_list=oxygen_entry[0].get('PaO2')
                 for PAO2_1 in PAO2_list:
                     if pd.Series((PAO2) >=PAO2_1.get('min')).all() and pd.Series((PAO2) <PAO2_1.get('max')).all():
                            score_ox=PAO2_1.get('points')
   
    elif pd.Series(FIO2 >=oxygen_entry[1].get('min')).all() and pd.Series((FIO2) < oxygen_entry[1].get('max')).all():
                g_list=oxygen_entry[1].get('A-a gradient')
                for g2 in g_list:
                    if pd.Series((gradient) >=g2.get('min')).all() and pd.Series((gradient) < g2.get('max')).all():
                            score_ox=g2.get('points')
                 
                 
    
    
    return score_ox

In [87]:
FIO2_PAO2_GRADIENT_CONFIG_FILE="apache-final.json"

In [88]:
(oxygen(61,59,458, FIO2_PAO2_GRADIENT_CONFIG_FILE))


3

In [97]:
oxygen(57, 74, 527, FIO2_PAO2_GRADIENT_CONFIG_FILE)

4

In [98]:
assert(oxygen(3.5, 20, 400, FIO2_PAO2_GRADIENT_CONFIG_FILE)==0)
assert(oxygen(66, 59, 205, FIO2_PAO2_GRADIENT_CONFIG_FILE)==2)
assert(oxygen(57, 74, 527, FIO2_PAO2_GRADIENT_CONFIG_FILE)==4)

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [99]:
###Test functions or assertions are above within each function. 

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [236]:
import json
"""
        This function combines all functions above to give you a total score of the parameters. The order is the same as the excel sheet provided.
        The test functions are below. 
"""
def apache_score2(risk_factors, age, temp, ph, heart, respiratory, sodium, potassium, creatinine, failure, hematocrit, white_blood_count, points, FIO2, PAO2, gradient):
    total_score=0
    total_score2=0
    
    
    config_file = "apache-final.json"
    total_score += risk_factors_score(risk_factors, config_file)
    total_score += all_factors(age, temp, ph, config_file)
    total_score += rates(heart, respiratory, config_file)
    total_score += serum(sodium, potassium, config_file)
    total_score += (muscle(creatinine, failure, config_file))
    total_score += blood(hematocrit,white_blood_count, config_file)
    total_score += glas(points)
    total_score += oxygen(FIO2, PAO2, gradient, config_file)
   
    total_score2=total_score+1
    
    
    return total_score2


In [196]:
(apache_score2(['Emergency'],6,27,7.3,174,10,170,7.8,1.6,"Chronic Renal Failure",3,28,13,61,59,458))

35

In [197]:
(apache_score2(['Nonoperative'],32,33,7.5,26,1,189,6,1.6,'Acute Renal Failure',64,8,1,45,73,424))

47

In [198]:
(apache_score2(['Nonoperative'],10,32,7.7,10,26,144,6.2,1.8,'Chronic Renal Failure',56,50,9,41,61,217))


34

In [199]:
(apache_score2(['Nonoperative'],62,28,7.1,56,7,177,0.4,1.6, "Acute Renal Failure",38,50,7,57,74,527))

47

### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [264]:
## last problem
import pandas as pd
import requests
import json 

site = 'https://hds5210-2020.s3.amazonaws.com/TestPatients.csv'
df=pd.read_csv(site)

#patients=pd.DataFrame(df)
patients=pd.read_csv(site,nrows=1)

finals =[]

for patient in patients:
    hist = patients.get('Organ Failure History')
    age1= patients.get('Age')
    temp1= patients.get('Temperature')
    ph1= patients.get('pH')
    lev= patients.get('Heart Rate')
    lev2= patients.get('Respiratory Rate')
    salt1= patients.get('Sodium')
    pot1= patients.get('Potassium')
    cre= patients.get('Creatinine')
    fail=patients.get('Acute Renal Failure')
    hemato1= patients.get('Hematocrit')
    blood2= patients.get('White Blood Count')
    coma= patients.get('Glasgow Coma Scale')
    fi= patients.get('FiO2')
    pa= patients.get('PaO2')
    gra= patients.get('A-a Gradient')
    
    #df["Age"]= df["Age"].astype(int)
    #df["Temperature"]= df["Temperature"].astype(int)
    #df["pH"]= df["pH"].astype(int)
    
    #risk_factors, age, temp, ph, heart, respiratory, sodium, potassium, creatinine, failure, hematocrit, white_blood_count, points, FIO2, PAO2, gradient
    score_end=apache_score2(hist,age1,temp1,ph1,lev,lev2,salt1,pot1,cre,fail, hemato1,blood2,coma,fi, pa, gra)
    
    #finals=score_end 
    
    print (score_end)

0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64
0    35
Name: Glasgow Coma Scale, dtype: int64


### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv
